In [1]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
#Erstellen von costum Retrieval Systemen

def create_index(documents, stemmer):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer=stemmer, stopwords=None)
    
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)



LemurKrovetzStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'LemurKrovetzStemmer')
WeakPorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'WeakPorterStemmer')
PorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'PorterStemmer')
EnglishSnowballStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'EnglishSnowballStemmer')
SnowballStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'SnowballStemmer')
TRv2WeakPorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'TRv2WeakPorterStemmer')
TRv2PorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'TRv2PorterStemmer')

XSqrA_M	 = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_LemurKrovetzStemmer	 = pt.BatchRetrieve(LemurKrovetzStemmer_index, wmodel="XSqrA_M")
XSqrA_M_WeakPorterStemmer	 = pt.BatchRetrieve(WeakPorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_PorterStemmer	 = pt.BatchRetrieve(PorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_EnglishSnowballStemmer	 = pt.BatchRetrieve(EnglishSnowballStemmer_index, wmodel="XSqrA_M")
XSqrA_M_SnowballStemmer	 = pt.BatchRetrieve(SnowballStemmer_index, wmodel="XSqrA_M")
XSqrA_M_TRv2WeakPorterStemmer	 = pt.BatchRetrieve(TRv2WeakPorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_TRv2PorterStemmer	 = pt.BatchRetrieve(TRv2PorterStemmer_index, wmodel="XSqrA_M")


term: nlp
term: have
term: often
term: been
term: performed
term: around
term: the
term: static
term: notion
term: of
term: words
term: where
term: the
term: context
term: surrounding
term: words
term: is
term: not
term: considered
term: for
term: example
term: previous
term: analyses
term: on
term: politeness
term: have
term: focused
term: on
term: comparing
term: the
term: use
term: of
term: static
term: words
term: such
term: as
term: personal
term: pronouns
term: across
term: im
term: polite
term: requests
term: without
term: taking
term: the
term: context
term: of
term: those
term: words
term: into
term: account
term: current
term: word
term: embeddings
term: in
term: nlp
term: do
term: capture
term: context
term: and
term: thus
term: can
term: be
term: leveraged
term: to
term: enrich
term: linguistic
term: analyses
term: in
term: this
term: work
term: we
term: introduce
term: a
term: model
term: which
term: leverages
term: the
term: pre
term: trained
term: bert
term: model
term:

: 

: 

In [2]:
#Evaluation

pt.Experiment(
    [XSqrA_M, XSqrA_M_TRv2PorterStemmer, XSqrA_M_TRv2WeakPorterStemmer, XSqrA_M_SnowballStemmer,XSqrA_M_EnglishSnowballStemmer, XSqrA_M_LemurKrovetzStemmer, XSqrA_M_PorterStemmer, XSqrA_M_WeakPorterStemmer],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = ["XSqrA_M", "XSqrA_M_TRv2PorterStemmer", "XSqrA_M_TRv2WeakPorterStemmer", "XSqrA_M_SnowballStemmer", "XSqrA_M_EnglishSnowballStemmer", "XSqrA_M_LemurKrovetzStemmer", "XSqrA_M_PorterStemmer", "XSqrA_M_WeakPorterStemmer"]
)

NameError: name 'XSqrA_M' is not defined